# CS231n Convulutional Neural Networks for Visual Recognition
http://cs231n.github.io/


In [1]:
import numpy as np

## Matrix Multiplication

1. Make sure that the the number of columns in the 1st one equals the number of rows in the 2nd one
2. Multiply the elements of each row of the first matrix by the elements of each column in the second matrix.
3. Add the products.
4. Output = #rows x #cols

In [6]:
x = np.matrix( ((2,3,4), (5,6,7)) ) 2x3
y = np.matrix( ((2, 3), (3,5), (6,7)) ) 3x2

x * y


matrix([[37, 49],
        [70, 94]])

## Linear classifier
http://cs231n.github.io/linear-classify/

One  interpretation for the weights WW is that each row of WW corresponds to a template (or sometimes also called a prototype) for one of the classes.

In [30]:
W = np.matrix( [[0.2, -0.5, 0.1, 2.0], #cat
                [1.5, 1.3, 2.1, 0.0],  #dog
               [0, 0.25, 0.2, -0.3]] )  #ship
x = np.matrix( [[56], 
                [231], 
                [24], 
                [2]] ) 

b = np.matrix ([[1.1], 
               [3.2],
               [-1.2]])

W*x + b


matrix([[ -96.8 ],
        [ 437.9 ],
        [  60.75]])

it is a little cumbersome to keep track of two sets of parameters (the biases bb and weights WW) separately. A commonly used trick is to combine the two sets of parameters into a single matrix that holds both of them by extending the vector xixi with one additional dimension that always holds the constant 1 - a default bias dimension. With the extra dimension, the new score function will simplify to a single matrix multiply.

In [28]:
W = np.matrix( [[0.2, -0.5, 0.1, 2.0, 1.1], #cat + bias
                [1.5, 1.3, 2.1, 0.0, 3.2],  #dog + bias
               [0, 0.25, 0.2, -0.3, -1.2]] ) #ship + bias
                 
x = np.matrix( [[56], 
                [231], 
                [24], 
                [2],
               [1]] ) #default bias 

W*x


matrix([[ -96.8 ],
        [ 437.9 ],
        [  60.75]])

## Linear Classification Demo
http://vision.stanford.edu/teaching/cs231n/linear-classify-demo/